In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

from collections import Counter
import pickle
import pandas as pd
import numpy as np

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, RandomizedSearchCV

In [ ]:
model = list()
resample = list()
precision = list()
recall = list()
F1score = list()
AUCROC = list()

def test_eval(clf_model, X_test, y_test, algo=None, sampling=None):
    # Test set prediction
    y_prob=clf_model.predict_proba(X_test)
    y_pred=clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n")
    print("macro F measure")
    print(f1_score(y_test,y_pred,average="macro"))
    print("weighted F measure")
    print(f1_score(y_test,y_pred,average="weighted"))
    print('AUC-ROC')
    print('='*60)
    print(roc_auc_score(y_test, y_prob[:,1]))

    model.append(algo)
    precision.append(precision_score(y_test,y_pred))
    recall.append(recall_score(y_test,y_pred))
    F1score.append(f1_score(y_test,y_pred))
    AUCROC.append(roc_auc_score(y_test, y_prob[:,1]))
    resample.append(sampling)
    report=classification_report(y_test,y_pred)
    report_path = "report_XGBoost.txt"
    text_file = open(report_path, "a")
    n = text_file.write(report)
    text_file.close()



In [ ]:
train_df =  pd.read_csv("Train_Set_Bot_Classification.csv")
del train_df['User Name']
#df=df.drop(['Sum of Length of Tweets','Min Length of Tweets','Max Length of Tweets','Sum of Length of Retweets','Min Length of Retweets','Max Length of Retweets','Sum of Length of Text',
#'Min Length of Text','Max Length of Text','No of Text','No of Text Containing Only URLs','No of Retweets Per Text','No of Digits in Text','No of Unique Mention','No of Mentioned Username',
#'No of Hashtag','No of Unique Text','No of URLs in Text','No of Mentions Per Text','Jaccard Similarity','Length of Screen Name','Length of User Name','Average of Length of Retweets','Average of Length of Text','Standard Deviation of Length of Retweets','Standard Deviation of Length of Text'],axis=1)
train_df.shape

train_df['Class'].value_counts()

0    2584
1     362
Name: Class, dtype: int64

In [ ]:
test_df =  pd.read_csv("Test_Set_Bot_Classification.csv")
del test_df['User Name']
del test_df['Class_y']
# Rename the column
test_df.rename(columns={'Class_x': 'Class'}, inplace=True)
#df=df.drop(['Sum of Length of Tweets','Min Length of Tweets','Max Length of Tweets','Sum of Length of Retweets','Min Length of Retweets','Max Length of Retweets','Sum of Length of Text',
#'Min Length of Text','Max Length of Text','No of Text','No of Text Containing Only URLs','No of Retweets Per Text','No of Digits in Text','No of Unique Mention','No of Mentioned Username',
#'No of Hashtag','No of Unique Text','No of URLs in Text','No of Mentions Per Text','Jaccard Similarity','Length of Screen Name','Length of User Name','Average of Length of Retweets','Average of Length of Text','Standard Deviation of Length of Retweets','Standard Deviation of Length of Text'],axis=1)
test_df.shape

test_df['Class'].value_counts()

0    647
1     90
Name: Class, dtype: int64

In [ ]:
X_train = train_df.drop(['Class'], axis=1)
y_train = train_df['Class']
X_test = test_df.drop(['Class'], axis=1)
y_test = test_df['Class']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler=scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# XGBoost

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=46)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

In [ ]:
from sklearn.metrics import f1_score, make_scorer


"""## Model-4: XGBoost

### 1. `Original Unsampled Data`
"""

from xgboost import XGBClassifier
xgb_model = XGBClassifier(reg_lambda = 12.8, reg_alpha =0.2, n_estimators = 2800, min_samples_split = 5, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 70, learning_rate = 0.06, gamma = 0.8)


xgb_model.fit(X_train, y_train)


test_eval(xgb_model, X_test, y_test, 'Random Forest', 'actual')

[07:29:08] WARNING: ../src/learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

Confusion Matrix
[[633  14]
 [ 38  52]] 

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       647
           1       0.79      0.58      0.67        90

    accuracy                           0.93       737
   macro avg       0.87      0.78      0.81       737
weighted avg       0.92      0.93      0.92       737
 

macro F measure
0.8136064744562468
weighted F measure
0.9246586765441657
AUC-ROC
0.8830156276833248


with full feature

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(reg_lambda = 12.8, reg_alpha =0.2, n_estimators = 2800, min_samples_split = 5, min_samples_leaf = 1, max_features = 'sqrt', max_depth = 70, learning_rate = 0.06, gamma = 0.8)

xgb_model.fit(X_train, y_train)


test_eval(xgb_model, X_test, y_test, 'Random Forest', 'actual')

[07:27:27] WARNING: ../src/learner.cc:767: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

Confusion Matrix
[[633  14]
 [ 38  52]] 

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       647
           1       0.79      0.58      0.67        90

    accuracy                           0.93       737
   macro avg       0.87      0.78      0.81       737
weighted avg       0.92      0.93      0.92       737
 

macro F measure
0.8136064744562468
weighted F measure
0.9246586765441657
AUC-ROC
0.8830156276833248


# Decision Tree

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=31)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [38] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
clf_DT =DecisionTreeClassifier(max_depth=10, max_features=0.4, min_samples_split=10,
                       random_state=10)
clf_DT.fit(X_train, y_train)


test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[627  20]
 [ 44  46]] 

Classification Report
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       647
           1       0.70      0.51      0.59        90

    accuracy                           0.91       737
   macro avg       0.82      0.74      0.77       737
weighted avg       0.91      0.91      0.91       737
 

macro F measure
0.7705925839461499
weighted F measure
0.9072722172851269
AUC-ROC
0.8076850420745321


In [ ]:
clf_DT = DecisionTreeClassifier(max_depth=10, max_features=0.4, min_samples_split=10,
                       random_state=10)
clf_DT.fit(X_train, y_train)


test_eval(clf_DT, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[628  19]
 [ 45  45]] 

Classification Report
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       647
           1       0.70      0.50      0.58        90

    accuracy                           0.91       737
   macro avg       0.82      0.74      0.77       737
weighted avg       0.91      0.91      0.91       737
 

macro F measure
0.767965367965368
weighted F measure
0.9066861677445124
AUC-ROC
0.7907178430362356


# Random Forest

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=43)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [38] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
clf_rf = RandomForestClassifier(class_weight='balanced_subsample', max_depth=20,
                       max_features='auto', min_samples_leaf=4,
                       min_samples_split=10, n_estimators=1600,
                       random_state=10)
clf_rf.fit(X_train, y_train)


test_eval(clf_rf, X_test, y_test, 'Decision Tree', 'actual')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Confusion Matrix
[[628  19]
 [ 36  54]] 

Classification Report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       647
           1       0.74      0.60      0.66        90

    accuracy                           0.93       737
   macro avg       0.84      0.79      0.81       737
weighted avg       0.92      0.93      0.92       737
 

macro F measure
0.8103119896299831
weighted F measure
0.9219654000777093
AUC-ROC
0.8801820367508159


with full feature

In [ ]:
clf_rf = RandomForestClassifier(class_weight='balanced_subsample', max_depth=110,
                       min_samples_leaf=4, min_samples_split=10,
                       n_estimators=2800, random_state=10)
clf_rf.fit(X_train, y_train)


test_eval(clf_rf, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[628  19]
 [ 36  54]] 

Classification Report
              precision    recall  f1-score   support

           0       0.95      0.97      0.96       647
           1       0.74      0.60      0.66        90

    accuracy                           0.93       737
   macro avg       0.84      0.79      0.81       737
weighted avg       0.92      0.93      0.92       737
 

macro F measure
0.8103119896299831
weighted F measure
0.9219654000777093
AUC-ROC
0.8835479993130689


# KNN

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=16)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [38] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)
test_eval(clf_knn, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[638   9]
 [ 42  48]] 

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.99      0.96       647
           1       0.84      0.53      0.65        90

    accuracy                           0.93       737
   macro avg       0.89      0.76      0.81       737
weighted avg       0.93      0.93      0.92       737
 

macro F measure
0.807314334927641
weighted F measure
0.9238936870495946
AUC-ROC
0.8413532543362527


with full feature

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)
test_eval(clf_knn, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[637  10]
 [ 48  42]] 

Classification Report
              precision    recall  f1-score   support

           0       0.93      0.98      0.96       647
           1       0.81      0.47      0.59        90

    accuracy                           0.92       737
   macro avg       0.87      0.73      0.77       737
weighted avg       0.92      0.92      0.91       737
 

macro F measure
0.7740028761155522
weighted F measure
0.9118952020991121
AUC-ROC
0.808517946075906


# SVM

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=33)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [38] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
from sklearn.svm import SVC

In [ ]:
clf_svm = SVC(C=400, class_weight='balanced', gamma=0.01, probability=True)
clf_svm.fit(X_train, y_train)
test_eval(clf_svm, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[578  69]
 [ 22  68]] 

Classification Report
              precision    recall  f1-score   support

           0       0.96      0.89      0.93       647
           1       0.50      0.76      0.60        90

    accuracy                           0.88       737
   macro avg       0.73      0.82      0.76       737
weighted avg       0.91      0.88      0.89       737
 

macro F measure
0.7630719011972347
weighted F measure
0.8869820746918591
AUC-ROC
0.8822600034346556


with full feature

In [ ]:
clf_svm = SVC(C=400, class_weight='balanced', gamma=0.01, probability=True)
clf_svm.fit(X_train, y_train)
test_eval(clf_svm, X_test, y_test, 'Decision Tree', 'actual')

Confusion Matrix
[[573  74]
 [ 22  68]] 

Classification Report
              precision    recall  f1-score   support

           0       0.96      0.89      0.92       647
           1       0.48      0.76      0.59        90

    accuracy                           0.87       737
   macro avg       0.72      0.82      0.75       737
weighted avg       0.90      0.87      0.88       737
 

macro F measure
0.7544561052806928
weighted F measure
0.8816132413214468
AUC-ROC
0.8775373518804741


# Logistic Regression

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression,f_classif
import sklearn.feature_selection as fs
bk = SelectKBest(f_classif, k=37)
bk=bk.fit(X_train, y_train)
X_train = bk.transform(X_train)
X_test = bk.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [38] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
clf_LR =  LogisticRegression(C=100, penalty='l1', solver='liblinear')
clf_LR.fit(X_train, y_train)
clf_LR.fit(X_train, y_train)
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'actual')

Confusion Matrix
[[635  12]
 [ 40  50]] 

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       647
           1       0.81      0.56      0.66        90

    accuracy                           0.93       737
   macro avg       0.87      0.77      0.81       737
weighted avg       0.92      0.93      0.92       737
 

macro F measure
0.8092801974679513
weighted F measure
0.9236922755800221
AUC-ROC
0.8720590760776232


with full feature

In [ ]:
clf_LR =  LogisticRegression(C=100, penalty='l1', solver='liblinear')
clf_LR.fit(X_train, y_train)
test_eval(clf_LR, X_test, y_test, 'Logistic Regression', 'actual')

Confusion Matrix
[[628  19]
 [ 41  49]] 

Classification Report
              precision    recall  f1-score   support

           0       0.94      0.97      0.95       647
           1       0.72      0.54      0.62        90

    accuracy                           0.92       737
   macro avg       0.83      0.76      0.79       737
weighted avg       0.91      0.92      0.91       737
 

macro F measure
0.7873302296948943
weighted F measure
0.9136014987340101
AUC-ROC
0.8701356688991929
